In [10]:
import pickle

with open('dataframe.pkl', 'rb') as file:
    df = pickle.load(file)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4646773 entries, 0 to 4646773
Data columns (total 27 columns):
 #   Column           Dtype         
---  ------           -----         
 0   FlagImpaye       category      
 1   Montant          float64       
 2   DateTransaction  datetime64[ns]
 3   CodeDecision     object        
 4   VerifianceCPT1   int32         
 5   VerifianceCPT2   int32         
 6   VerifianceCPT3   int32         
 7   D2CB             int32         
 8   ScoringFP1       float64       
 9   ScoringFP2       float64       
 10  ScoringFP3       float64       
 11  TauxImpNb_RB     float64       
 12  TauxImpNB_CPM    float64       
 13  EcartNumCheq     int32         
 14  NbrMagasin3J     int32         
 15  DiffDateTr1      float64       
 16  DiffDateTr2      float64       
 17  DiffDateTr3      float64       
 18  CA3TRetMtt       float64       
 19  CA3TR            float64       
 20  Heure            int32         
 21  Year             int32         
 22 

### On définit la partie Train et Test

In [12]:
# Définition des dates limites pour les ensembles d'apprentissage et de test
train_inf = '2017-02-01'
train_sup = '2017-08-31'
test_inf = '2017-09-01'
test_sup = '2017-11-30'

In [13]:
train = df.loc[(df['DateTransaction'] >= train_inf) & (df['DateTransaction'] <= train_sup)]
X_train = train.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_train = train['FlagImpaye']

In [14]:
test = df.loc[(df['DateTransaction'] >= test_inf) & (df['DateTransaction'] <= test_sup)]
X_test = test.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_test = test['FlagImpaye']

In [15]:
len(X_train)

3888468

In [16]:
len(X_test)

737068

## Test de chaque méthode d'échantillonage par modeles

In [17]:
from sklearn import metrics
from sklearn.metrics import f1_score

In [18]:
!pip install 'imblearn'
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE

ERROR: Invalid requirement: "'imblearn'"

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install xgboost


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


On fait tourner DecisionTreeClassifier, RandomForestClassifier, GradientBoostingClassifier,KNeighborsClassifier, SVC, adaboost et xgboost

In [20]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from tqdm import tqdm

In [12]:
# Définir les stratégies d'échantillonnage
sampling_strategies = [0.05, 0.1]  # Ajoutez les valeurs que vous souhaitez tester

#on garde deux oversampling et deux undersampling
samplers = [RandomOverSampler, SMOTE, RandomUnderSampler, NearMiss]

# Précalculer les échantillons resamplés pour chaque méthode d'échantillonnage
resampled_data = {}
for sampler in tqdm(samplers, desc="Resampling Progress"):
    for strategy in tqdm(sampling_strategies, desc=f"{sampler.__name__} Progress", leave=False):
        key = (sampler, strategy)
        X_resampled, y_resampled = sampler(sampling_strategy=strategy).fit_resample(X_train, y_train)
        resampled_data[key] = (X_resampled, y_resampled)

Resampling Progress: 100%|██████████| 4/4 [02:52<00:00, 43.12s/it]


In [29]:
# Définir les modèles à tester
models = [
    DecisionTreeClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_1 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_1 = pd.concat([results_df_1, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_1)
results_df_1.to_pickle('dataframe_recap_sampling_1.pkl')

C:\Users\marevel\AppData\Local\Temp\ipykernel_14404\260053566.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_1 = pd.concat([results_df_1, pd.DataFrame({


Model: DecisionTreeClassifier, Sampler: RandomOverSampler, Strategy: 0.05, F1 Score: 0.030195815287012748
Model: DecisionTreeClassifier, Sampler: RandomOverSampler, Strategy: 0.1, F1 Score: 0.026262189853104554
Model: DecisionTreeClassifier, Sampler: SMOTE, Strategy: 0.05, F1 Score: 0.03714936856354632
Model: DecisionTreeClassifier, Sampler: SMOTE, Strategy: 0.1, F1 Score: 0.03639382151402204
Model: DecisionTreeClassifier, Sampler: RandomUnderSampler, Strategy: 0.05, F1 Score: 0.03486636000379733
Model: DecisionTreeClassifier, Sampler: RandomUnderSampler, Strategy: 0.1, F1 Score: 0.03289138545840045
Model: DecisionTreeClassifier, Sampler: NearMiss, Strategy: 0.05, F1 Score: 0.02929911735260891
Model: DecisionTreeClassifier, Sampler: NearMiss, Strategy: 0.1, F1 Score: 0.025979530864366336
                    Model             Sampler  Sampling Strategy  F1 Score
0  DecisionTreeClassifier   RandomOverSampler               0.05  0.030196
1  DecisionTreeClassifier   RandomOverSampler      

In [34]:

# Définir les modèles à tester
models = [
    RandomForestClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_2 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_2 = pd.concat([results_df_2, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_2)
results_df_2.to_pickle('dataframe_recap_sampling_2.pkl')

Model: RandomForestClassifier, Sampler: NearMiss, Strategy: 0.1, F1 Score: 0.02952237759137063
                    Model   Sampler  Sampling Strategy  F1 Score
0  RandomForestClassifier  NearMiss                0.1  0.029522


C:\Users\marevel\AppData\Local\Temp\ipykernel_14404\4242767768.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_2_bis = pd.concat([results_df_2_bis, pd.DataFrame({


In [13]:

# Définir les modèles à tester
models = [
    SVC()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_3 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_3 = pd.concat([results_df_3, pd.DataFrame({
                'Model': [model.__class__.__name__],
                
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_3)
results_df_3.to_pickle('dataframe_recap_sampling_3.pkl')

In [ ]:
# Définir les modèles à tester
models = [
    AdaBoostClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_4 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_4 = pd.concat([results_df_4, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_4)
results_df_4.to_pickle('dataframe_recap_sampling_4.pkl')

In [ ]:
# Définir les modèles à tester
models = [
    XGBClassifier()
]

# Initialiser un DataFrame pour stocker les résultats
results_df_5 = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Récupérer les échantillons resamplés précalculés
            X_resampled, y_resampled = resampled_data[(sampler, strategy)]

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df_5 = pd.concat([results_df_5, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)

            # Afficher le message de progression
            tqdm.write(f"Model: {model.__class__.__name__}, Sampler: {sampler.__name__}, Strategy: {strategy}, F1 Score: {f1}")

# Afficher le tableau récapitulatif
print(results_df_5)
results_df_5.to_pickle('dataframe_recap_sampling_5.pkl')

On réunit les résultats dans un seul tableau recapitulatif

In [3]:
results_df_1 = pd.read_pickle('dataframe_recap_sampling_1.pkl')
results_df_2 = pd.read_pickle('dataframe_recap_sampling_2.pkl')
#results_df_3 = pd.read_pickle('dataframe_recap_sampling_3.pkl')
results_df_4 = pd.read_pickle('dataframe_recap_sampling_4.pkl')
results_df_5 = pd.read_pickle('dataframe_recap_sampling_5.pkl')

In [5]:
#results_df = pd.concat([results_df_1, results_df_2, results_df_3, results_df_4, results_df_5], ignore_index=True)
results_df = pd.concat([results_df_1, results_df_2, results_df_4, results_df_5], ignore_index=True)
results_df.to_pickle('dataframe_recap_sampling.pkl')

In [6]:
results_df.describe(include='all')

,Model,Sampler,Sampling Strategy,F1 Score
count,32,32,32.0000,32.000000
unique,4,4,NaN,NaN
top,DecisionTreeClassifier,RandomOverSampler,NaN,NaN
freq,8,8,NaN,NaN
mean,NaN,NaN,0.0750,0.056036
std,NaN,NaN,0.0254,0.032969
min,NaN,NaN,0.0500,0.015576
25%,NaN,NaN,0.0500,0.031354
50%,NaN,NaN,0.0750,0.038366
75%,NaN,NaN,0.1000,0.072764


In [7]:
# Grouper les résultats par modèle
grouped_results = results_df.groupby('Model')

# Initialiser un DataFrame pour stocker les meilleurs résultats par modèle
best_results_df = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque groupe
for model, group_df in grouped_results:
    # Trouver la ligne avec le plus grand F1 score pour chaque modèle
    best_result = group_df.sort_values(by='F1 Score', ascending=False).iloc[0]
    
    # Ajouter le meilleur résultat au DataFrame final
    # Remplacez la ligne best_results_df = best_results_df.append(...) par ceci
    best_results_df = pd.concat([best_results_df, pd.DataFrame({
        'Model': [best_result['Model']],
        'Sampler': [best_result['Sampler']],
        'Sampling Strategy': [best_result['Sampling Strategy']],
        'F1 Score': [best_result['F1 Score']]
    })], ignore_index=True)

# Afficher le tableau récapitulatif des meilleurs résultats pour chaque modèle
print(best_results_df)
best_results_df.to_pickle('best_results_df.pkl')

                    Model             Sampler  Sampling Strategy  F1 Score
0      AdaBoostClassifier   RandomOverSampler               0.10  0.131428
1  DecisionTreeClassifier               SMOTE               0.05  0.037149
2  RandomForestClassifier  RandomUnderSampler               0.05  0.102525
3           XGBClassifier               SMOTE               0.10  0.077850


C:\Users\marevel\AppData\Local\Temp\ipykernel_16984\346220120.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  best_results_df = pd.concat([best_results_df, pd.DataFrame({


In [8]:
best_results_df

,Model,Sampler,Sampling Strategy,F1 Score
0,AdaBoostClassifier,RandomOverSampler,0.10,0.131428
1,DecisionTreeClassifier,SMOTE,0.05,0.037149
2,RandomForestClassifier,RandomUnderSampler,0.05,0.102525
3,XGBClassifier,SMOTE,0.10,0.077850


## Maintenant qu'on a une vue d'ensemble claire sur les méthodes de re-échantillonage qui marchent le mieux en fonction des modèles, on va reprendre ces combinaisons pour réaliser une optimisation cette fois-ci des hyper-paramètres des modèles à l'aide d'un greedsearch.

### Model: GradientBoostingClassifier - Sampler: RandomUnderSampler - Sampling Strategy: 0.1

In [22]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
import pickle


# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.10)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle de l'AdaBoostClassifier
adab_classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())  # Utilise un arbre de décision faible par défaut

# Pipeline
pipeline = Pipeline([
    ('classifier', adab_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__base_estimator__max_depth': [1, 2, 3],  # Ajustez ici pour le paramètre de l'arbre faible
    'classifier__learning_rate': [0.1, 0.5, 1.0],
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

with open('modele_adaboost.pkl', 'wb') as file:
    pickle.dump(grid_search, file)

c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Sous-échantillonnage
smote = SMOTE(sampling_strategy=0.10)
X_train_rus, y_train_rus = smote.fit_resample(X_train, y_train)

# Modèle du RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Pipeline
pipeline_rf = Pipeline([
    ('classifier', rf_classifier)
])

tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid_rf = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [None, 10, 20],  # Ajustez ici pour le paramètre de profondeur
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
}

# Recherche sur la grille
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, scoring='f1', cv=tscv)
grid_search_rf.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres (Random Forest):", grid_search_rf.best_params_)
print("Meilleur score F1 (Random Forest):", grid_search_rf.best_score_)

with open('modele_random_forest.pkl', 'wb') as file_rf:
    pickle.dump(grid_search_rf, file_rf)


In [ ]:
from xgboost import XGBClassifier

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.10)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle du XGBClassifier
xgb_classifier = XGBClassifier()

# Pipeline
pipeline_xgb = Pipeline([
    ('classifier', xgb_classifier)
])

tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid_xgb = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [3, 5, 7],  # Ajustez ici pour le paramètre de profondeur
    'classifier__learning_rate': [0.1, 0.01, 0.001],
    'classifier__subsample': [0.8, 1.0],
    'classifier__colsample_bytree': [0.8, 1.0],
}

# Recherche sur la grille
grid_search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, scoring='f1', cv=tscv)
grid_search_xgb.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres (XGBoost):", grid_search_xgb.best_params_)
print("Meilleur score F1 (XGBoost):", grid_search_xgb.best_score_)

with open('modele_xgboost.pkl', 'wb') as file_xgb:
    pickle.dump(grid_search_xgb, file_xgb)


In [35]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.1)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train, y_train)

 
# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

Meilleurs paramètres: {'classifier__max_depth': 15, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
Meilleur score F1: 0.10323029387708163


In [17]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.1)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train_ros, y_train_ros)

 
# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

In [44]:
import numpy as np

# Compter le nombre d'éléments uniques dans y_train_rus
unique_classes, counts = np.unique(y_train_rus, return_counts=True)

# Afficher le nombre d'éléments uniques et leurs occurrences
for cls, count in zip(unique_classes, counts):
    print(f"Classe {cls}: {count} occurrences")

Classe 0: 2290 occurrences
Classe 1: 229 occurrences


In [33]:
len(y_train)

38827

In [34]:
len(y_train_rus)

2519

In [30]:

# Utiliser le meilleur modèle trouvé par la recherche sur la grille
best_model = grid_search.best_estimator_

# Faire des prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Calculer le F1-score sur l'ensemble de test
f1_test = f1_score(y_test, y_pred)

# Afficher le F1-score sur l'ensemble de test
print("F1 Score sur l'ensemble de test:", f1_test)


F1 Score sur l'ensemble de test: 0.054590570719602965
